<a href="https://colab.research.google.com/github/YuLiu83/Practical-business-case-code-demo/blob/main/Text_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Summarization:


Using 'TextRank' based algorithm to create text summarizations for large financial documents, the technique is extremely useful when trying to pull insights from large volume of long written documents. 



### Load and Observe Data:

In [ ]:
# Mount google drive
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# load data 
import pickle
with open('/content/drive/My Drive/Data/SummarizationData.pkl', 'rb') as file:
    # read the data as binary data stream
    dataset = pickle.load(file)

In [ ]:
# observe data
print('Shape: {}'.format(dataset.shape))
print(dataset.columns)

Shape: (5, 2)
Index(['text', 'DocumentId'], dtype='object')


In [ ]:
# Sort documents in order
dataset=dataset.sort_values(['DocumentId'])

There are five text documents from raw data.

In [ ]:
# example
dataset['text'][0]

"\ufeff\n\nITEM 7. Management’s Discussion and Analysis of Financial Condition and Results of Operations\n\nThis Management’s Discussion and Analysis of Financial Condition and Results of Operations and other parts of this current report contain forward-\nlooking statements that involve risks and uncertainties. Forward- looking statements can also be identified by words such as “anticipates,” “expects,”\n“believes,” “plans,” “predicts,” and similar terms. Forward- looking statements are not guarantees of future performance and our actual results may\ndiffer significantly from the results discussed in the forward- looking statements. Factors that might cause such differences include but are not limited\nto those discussed in the subsection entitled Forward- Looking Statements and Factors That May Affect Future Results and Financial Condition\nbelow. The following discussion should be read in conjunction with our financial statements and notes thereto included in this current report. Our

In [ ]:
from bs4 import BeautifulSoup #xlml
BeautifulSoup(dataset['text'][0], 'html.parser')

﻿

ITEM 7. Management’s Discussion and Analysis of Financial Condition and Results of Operations

This Management’s Discussion and Analysis of Financial Condition and Results of Operations and other parts of this current report contain forward-
looking statements that involve risks and uncertainties. Forward- looking statements can also be identified by words such as “anticipates,” “expects,”
“believes,” “plans,” “predicts,” and similar terms. Forward- looking statements are not guarantees of future performance and our actual results may
differ significantly from the results discussed in the forward- looking statements. Factors that might cause such differences include but are not limited
to those discussed in the subsection entitled Forward- Looking Statements and Factors That May Affect Future Results and Financial Condition
below. The following discussion should be read in conjunction with our financial statements and notes thereto included in this current report. Our
fiscal year en

### Data processing:

In [ ]:
# remove speical characters
import re
pattern = r'\s|\ufeff'
dataset['text_cleaned']=[re.sub(pattern, ' ', text) for text in dataset['text']]

In [ ]:
# lemmetize for key word extraction
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('punkt')

lmtzr=WordNetLemmatizer()
Data_Lemmatized=[]
holder=[]

dataset['text_tokenized']=[nltk.word_tokenize(text) for text in dataset['text_cleaned']]

for i in dataset['text_tokenized']:
  holder=[]
  for j in i:

    holder.append(lmtzr.lemmatize(j.lower(), wn.NOUN))
  Data_Lemmatized.append(holder)

dataset['Lemmatized']=[' '.join(text) for text in Data_Lemmatized]

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Summarization and Keyword Extraction:

In [ ]:
from gensim.summarization import summarize
from gensim.summarization import keywords

dataset['summary']=[summarize(text, ratio=0.1)  for text in dataset['text_cleaned']]
dataset['keyword']=[keywords(text, ratio=0.1).replace('\n', ', ') for text in dataset['Lemmatized']]


First document:

In [ ]:
print('Summary: {}'.format(BeautifulSoup(dataset['summary'][0], 'html.parser')))
print('')
print('Key words: {}'.format(dataset['keyword'][0]))

# dataset.loc[dataset['DocumentId']==0, 'summary'].values[0]

Summary: Mobetize’s financial condition, results of operations and the carrying value of its technology depends on its ability to develop, market and sell innovative products and Services to FinTech Customers.
Our auditors have issued a going concern opinion on the financial statements for the year ended March 31, 2018, which indicates that the continuation of Mobetize as a going concern is dependent on continued financial support from its management and its ability to obtain necessary debt or equity financing, increase revenue, and generate profitable operations.
RESULTS OF OPERATIONS  During the twelve months ended March 31, 2018, Mobetize (i) continued to develop its FinTech products and Services; (ii) entered into certain development and commercialization agreements; (iii) acquired certain technology; (iv) secured debt and equity financing; and (v) satisfied continuous public disclosure requirements.
Despite our projection that operating expenses will increase over the next twelve 

Second document:

In [ ]:
print('Summary: {}'.format(BeautifulSoup(dataset['summary'][1], 'html.parser')))
print('')
print('Key words: {}'.format(dataset['keyword'][1]))

Summary: MANAGEMENT'S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF OPERATIONS  Overview and Background  We are engaged in the development and sale of renewable energy solutions using our direct methanol micro fuel cell technology, our reformer technology, and our silicon based rechargeable battery technology.
Since inception, we have reported net losses, including losses of $2,418,835 and $4,363,574 during the years ended September 30, 2016 and 2015, respectively, and we expect losses to continue in the near future as we grow our operations.
Table of Contents Liquidity and Capital Resources  11  We used cash of $878,548 in our operating activities in the year ended September 30, 2016, compared to $1,956,157 in the same period in 2015.
During the year ended September 30, 2016, we incurred changes in operating assets and liabilities of $776,456 that off set our use of cash.
We used $0 for investing activities related to fixed asset purchases and received $61,497 in proce



Third document:

In [ ]:
print('Summary: {}'.format(BeautifulSoup(dataset['summary'][2], 'html.parser')))
print('')
print('Key words: {}'.format(dataset['keyword'][2]))

Summary: These operating cash flows are largely dependent on such factors as oil and gas prices and production volumes, which are influenced by many factors beyond the control of the Trust.
The Monthly Income Amount is an amount equal to the sum of cash received by the Trust during a monthly period (the period commencing on the day after a monthly record date and continuing through and including the next succeeding monthly record date) attributable to the Royalty Properties, any reduction in cash reserves and any other cash receipts of the Trust, including interest, reduced by the sum of liabilities paid and any increase in cash reserves.
Results of Operations Distributable income consists of royalty income plus interest income plus any decrease in cash reserves established by the Trustee less general and administrative expenses of the Trust less any increase in cash reserves established by the Trustee.
The Trust's royalty income represents payments received during a particular time pe



Forth document:

In [ ]:
print('Summary: {}'.format(BeautifulSoup(dataset['summary'][3], 'html.parser')))
print('')
print('Key words: {}'.format(dataset['keyword'][3]))

Summary: As a result of the current difficult economic environment and our lack of funding to implement our business plan, during our fiscal year ended October 31, 2016, our Board of Directors began to analyze strategic alternatives available to our Company to continue as a going concern.
Pursuant to the terms of the proposed Merger, we will acquire all of the issued and outstanding stock of NewGen New Jersey in exchange for the issuance to the existing stockholders of NewGen New Jersey of 40,000,000 shares of our common stock.
As set forth in the Merger Agreement, we will acquire all of the issued and outstanding capital stock of NewGen New Jersey in exchange for the issuance to the current holders thereof of 40,000,000 shares of our common stock.
If we are able to close the Merger transaction as contemplated by the Merger Agreement, NewGen New Jersey will become our wholly-owned subsidiary, and we will report such event on a Current Report on Form 8-K to be filed with the SEC.
The in



Fifth document:

In [ ]:
print('Summary: {}'.format(BeautifulSoup(dataset['summary'][4], 'html.parser')))
print('')
print('Key words: {}'.format(dataset['keyword'][4]))

Summary: In October 2012, this segment was discontinued through the sale of substantially all of the assets of Bulova Technologies Ordnance Systems LLC, with any remaining assets and liabilities associated with that operation being segregated and reported as a discontinued operation, until March 29, 2017, when the Company sold all of its ownership interests in Ordnance.
Operating expenses and interest for the discontinued operations of Ordnance for the year ended September 30, 2017 of $33,385 is a decrease of $458,168 when compared to operating expenses and interest for the year ended September 30, 2016 of $491,553, and is due primarily to legal costs, as well as additional interest costs incurred in the restructuring of the terms of certain debts associated with the discontinued operations of Ordnance.
Continuing Operations  Revenue for continuing operations for the year ended September 30, 2017 of $25,150,112 is an increase of $6,429,350 when compared to the revenue for the year ende

### Appendix:

TextRank is an extractive and unsupervised text summarization techniqu:

*   The first step would be to concatenate all the text contained in the articles
*   Then split the text into individual sentences
*   In the next step, we will find vector representation (word embeddings) for each and every sentence
*   Similarities between sentence vectors are then calculated and stored in a matrix
*   The similarity matrix is then converted into a graph, with sentences as vertices and similarity scores as edges, for sentence rank calculation-- i.e. finding the most important sentence in the graph
*   Finally, a certain number of top-ranked sentences form the final summary






